In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import streamlit as st

pd.set_option('display.max_columns', None)

# Carrega o DataFrame do seu arquivo Excel
df = pd.read_excel('dados0408.xlsx')

from datetime import datetime

now = datetime.now()

colunas_selecionadas = ['FILIAL', 'PEDIDO', 'DATA EMISSAO PEDIDO', 'HORA EMISSAO PEDIDO', 'DATA ASS REMESSA', 'HORA ASS REMESSA',
                        'DATA PREPARACAO DO ITEM', 'HORA PREPARACAO DO ITEM', 'DATA GERACAO DA NOTA FISCAL', 'HORA GERACAO DA NOTA FISCAL',
                        'DATA GERACAO DO REGISTRO', 'HORA GERACAO DO REGISTRO'] 

df_trabalho = df[colunas_selecionadas].copy()

In [18]:
df_trabalho['DATA EMISSAO PEDIDO'] = pd.to_datetime(df_trabalho['DATA EMISSAO PEDIDO'], errors='coerce', dayfirst=True)
df_trabalho['HORA EMISSAO PEDIDO'] = pd.to_datetime(df['HORA EMISSAO PEDIDO'], format='%H:%M:%S', errors='coerce').dt.time
df_trabalho['DATA ASS REMESSA'] = pd.to_datetime(df_trabalho['DATA ASS REMESSA'], errors='coerce', dayfirst=True)
df_trabalho['HORA ASS REMESSA'] = pd.to_datetime(df['HORA ASS REMESSA'], format='%H:%M:%S', errors='coerce').dt.time
df_trabalho['DATA PREPARACAO DO ITEM'] = pd.to_datetime(df_trabalho['DATA PREPARACAO DO ITEM'], errors='coerce', dayfirst=True)
df_trabalho['HORA PREPARACAO DO ITEM'] = pd.to_datetime(df_trabalho['HORA PREPARACAO DO ITEM'], format='%H:%M:%S', errors='coerce').dt.time
df_trabalho['DATA GERACAO DO REGISTRO'] = pd.to_datetime(df_trabalho['DATA GERACAO DO REGISTRO'], errors='coerce', dayfirst=True)
df_trabalho['HORA GERACAO DO REGISTRO'] = pd.to_datetime(df_trabalho['HORA GERACAO DO REGISTRO'], format='%H:%M:%S', errors='coerce').dt.time

In [19]:
# Função auxiliar para combinar data e hora, lidando com nulos
def combinar_data_hora(data, hora):
    if pd.isna(data) or pd.isna(hora): # Verifica se data OU hora são nulas
        return pd.NaT # Retorna NaT (Not a Time) se alguma for nula
    try:
        # Combina o ano, mês, dia da data com hora, minuto, segundo da hora
        return pd.to_datetime(f"{data.year}-{data.month}-{data.day} {hora.hour}:{hora.minute}:{hora.second}")
    except:
        return pd.NaT # Retorna NaT se houver algum erro inesperado na combinação


In [34]:
df_trabalho['TIMESTAMP PEDIDO'] = df_trabalho.apply(lambda row: combinar_data_hora(row['DATA EMISSAO PEDIDO'], row['HORA EMISSAO PEDIDO']), axis=1)
df_trabalho['TIMESTAMP REMESSA'] = df_trabalho.apply(lambda row: combinar_data_hora(row['DATA ASS REMESSA'], row['HORA ASS REMESSA']), axis=1)
df_trabalho['TIMESTAMP ITEM'] = df_trabalho.apply(lambda row: combinar_data_hora(row['DATA PREPARACAO DO ITEM'], row['HORA PREPARACAO DO ITEM']), axis=1)
df_trabalho['TIMESTAMP TITULO'] = df_trabalho.apply(lambda row: combinar_data_hora(row['DATA GERACAO DO REGISTRO'], row['HORA GERACAO DO REGISTRO']), axis=1)

print(df_trabalho[['PEDIDO','TIMESTAMP PEDIDO', 'TIMESTAMP REMESSA', 'TIMESTAMP ITEM', 'TIMESTAMP TITULO']].head())

   PEDIDO    TIMESTAMP PEDIDO   TIMESTAMP REMESSA      TIMESTAMP ITEM  \
0    3305 2025-04-08 12:47:00 2025-04-08 12:50:00 2025-04-08 14:56:00   
1    3305 2025-04-08 12:47:00 2025-04-08 12:50:00 2025-04-08 14:56:00   
2    3305 2025-04-08 12:47:00 2025-04-08 12:50:00 2025-04-08 14:56:00   
3    3305 2025-04-08 12:47:00 2025-04-08 12:50:00 2025-04-08 14:56:00   
4    3305 2025-04-08 12:47:00 2025-04-08 12:50:00 2025-04-08 14:56:00   

     TIMESTAMP TITULO  
0 2025-04-08 16:08:00  
1 2025-04-08 16:08:00  
2 2025-04-08 16:08:00  
3 2025-04-08 16:08:00  
4 2025-04-08 16:08:00  


In [35]:
# Lista das colunas de timestamp para facilitar o tratamento
timestamp_cols = ['TIMESTAMP PEDIDO', 'TIMESTAMP REMESSA', 'TIMESTAMP ITEM', 'TIMESTAMP TITULO']

for col in timestamp_cols:
    # Converte a coluna para o tipo datetime, permitindo que datas inválidas se tornem NaT
    df_trabalho[col] = pd.to_datetime(df_trabalho[col], errors='coerce')
    
    # Substitui os valores com ano 1900 por NaT
    df_trabalho.loc[df_trabalho[col].dt.year == 1900, col] = np.nan
    
    # Substitui valores nulos (NaN, NaT) por None, que é a representação de ausência no Python
    df_trabalho[col] = df_trabalho[col].apply(lambda x: None if pd.isna(x) else x)

In [36]:
# Cria um dicionário para mapear as colunas de timestamp para as novas colunas de status
status_cols_map = {
    'TIMESTAMP PEDIDO': 'STATUS PEDIDO',
    'TIMESTAMP REMESSA': 'STATUS REMESSA',
    'TIMESTAMP ITEM': 'STATUS ITEM',
    'TIMESTAMP TITULO': 'STATUS TITULO'
}

# Itera sobre cada coluna de timestamp e cria a coluna de status correspondente
for ts_col, status_col in status_cols_map.items():
    # Usa np.where para criar a coluna de status
    # Se o valor na coluna de timestamp não for nulo (isnull() == False), o status é 'Concluído'
    # Caso contrário, o status é 'Pendente'
    df_trabalho[status_col] = df_trabalho[ts_col].notnull().apply(
        lambda x: 'Concluído' if x else 'Pendente'
    )

# Visualiza o resultado para as colunas de timestamp e status
print(df_trabalho[['PEDIDO', 'STATUS PEDIDO','STATUS REMESSA','STATUS ITEM','STATUS TITULO']].tail())

      PEDIDO STATUS PEDIDO STATUS REMESSA STATUS ITEM STATUS TITULO
6670  672050     Concluído       Pendente    Pendente      Pendente
6671  687350     Concluído      Concluído    Pendente      Pendente
6672  686913     Concluído      Concluído    Pendente      Pendente
6673  687017     Concluído      Concluído    Pendente      Pendente
6674  192793     Concluído       Pendente    Pendente      Pendente
